In [ ]:
!pip install scipy==1.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.insert(0,'/content/drive/My Drive/CS523/Project/face_classification/src')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/CS523/Project/face_classification/src

In [ ]:
import cv2
#from keras.models import load_model
from keras.models import *
#if having problem with importing, try to rerun this cell several times
import numpy as np

from utils.datasets import get_labels
from utils.inference import detect_faces
from utils.inference import draw_text
from utils.inference import draw_bounding_box
from utils.inference import apply_offsets
from utils.inference import load_detection_model
from utils.inference import load_image
from utils.preprocessor import preprocess_input
from datetime import datetime

In [ ]:
# parameters for loading data and images
#image_path = sys.argv[1]
#choose the model you want to use, change the path if needed.
image_path = '../images/Female_Emotions.jpeg'
detection_model_path = '../trained_models/detection_models/haarcascade_frontalface_default.xml'
emotion_model_path = '../trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5'
# emotion_model_path = '../trained_models/model_training_experiments/fer2013_mini_XCEPTION.74-0.66.hdf5'
# emotion_model_path = '../trained_models/reduced_modules_experiments_2/fer2013_mini_XCEPTION.100-0.51.hdf5'
# emotion_model_path = '../trained_models/reduced_modules_experiments_3/fer2013_mini_XCEPTION.93-0.53.hdf5'
# emotion_model_path = '../trained_models/increased_modules_experiments_5/fer2013_mini_XCEPTION.60-0.66.hdf5'
emotion_labels = get_labels('fer2013')
gender_labels = get_labels('imdb')
font = cv2.FONT_HERSHEY_SIMPLEX

In [ ]:
# hyper-parameters for bounding boxes shape
gender_offsets = (30, 60)
gender_offsets = (10, 10)
emotion_offsets = (20, 40)
emotion_offsets = (0, 0)

In [ ]:
# loading models
face_detection = load_detection_model(detection_model_path)
gender_classifier = load_model(gender_model_path, compile=False)

In [ ]:
%%time
emotion_classifier = load_model(emotion_model_path, compile=False)

CPU times: user 322 ms, sys: 5.42 ms, total: 327 ms
Wall time: 985 ms


In [ ]:
# getting input model shapes for inference
emotion_target_size = emotion_classifier.input_shape[1:3]
gender_target_size = gender_classifier.input_shape[1:3]

In [ ]:
emotion_target_size

(64, 64)

In [ ]:
gender_target_size

(48, 48)

In [ ]:
# loading images
rgb_image = load_image(image_path, grayscale=False)
gray_image = load_image(image_path, grayscale=True)
gray_image = np.squeeze(gray_image)
gray_image = gray_image.astype('uint8')

In [ ]:
faces = detect_faces(face_detection, gray_image)
for face_coordinates in faces:
    x1, x2, y1, y2 = apply_offsets(face_coordinates, gender_offsets)
    rgb_face = rgb_image[y1:y2, x1:x2]

    x1, x2, y1, y2 = apply_offsets(face_coordinates, emotion_offsets)
    gray_face = gray_image[y1:y2, x1:x2]

    try:
        rgb_face = cv2.resize(rgb_face, (gender_target_size))
        gray_face = cv2.resize(gray_face, (emotion_target_size))
    except:
        continue

    rgb_face = preprocess_input(rgb_face, False)
    rgb_face = np.expand_dims(rgb_face, 0)
    gender_prediction = gender_classifier.predict(rgb_face)
    gender_label_arg = np.argmax(gender_prediction)
    gender_text = gender_labels[gender_label_arg]

    gray_face = preprocess_input(gray_face, True)
    gray_face = np.expand_dims(gray_face, 0)
    gray_face = np.expand_dims(gray_face, -1)
    #print(gray_face)
    # print(gray_face.shape)
    # print(gray_face[0])
    ##
    start_time = datetime.now()

    emotion_label_arg = np.argmax(emotion_classifier.predict(gray_face))
    ##
    end_time = datetime.now()
    #print('Duration: {}'.format(end_time - start_time))

    emotion_text = emotion_labels[emotion_label_arg]

    if gender_text == gender_labels[0]:
        color = (0, 0, 255)
    else:
        color = (255, 0, 0)

    draw_bounding_box(face_coordinates, rgb_image, color)
    draw_text(face_coordinates, rgb_image, gender_text, color, 0, -20, 1, 2)
    draw_text(face_coordinates, rgb_image, emotion_text, color, 0, -50, 1, 2)

(1, 64, 64, 1)
[[[-0.56078434]
  [-0.8039216 ]
  [-0.8901961 ]
  ...
  [-0.6627451 ]
  [-0.52156866]
  [-0.44313723]]

 [[-0.8901961 ]
  [-0.8901961 ]
  [-0.8980392 ]
  ...
  [-0.6784314 ]
  [-0.6862745 ]
  [-0.6392157 ]]

 [[-0.96862745]
  [-0.9529412 ]
  [-0.94509804]
  ...
  [-0.73333335]
  [-0.70980394]
  [-0.6392157 ]]

 ...

 [[-0.90588236]
  [-0.92156863]
  [-0.9372549 ]
  ...
  [-0.7882353 ]
  [-0.8980392 ]
  [-0.92941177]]

 [[-0.96862745]
  [-0.94509804]
  [-0.92156863]
  ...
  [-0.84313726]
  [-0.9372549 ]
  [-0.9607843 ]]

 [[-0.92941177]
  [-0.92156863]
  [-0.8901961 ]
  ...
  [-0.8980392 ]
  [-0.9607843 ]
  [-0.99215686]]]
(1, 64, 64, 1)
[[[-0.8117647 ]
  [-0.75686276]
  [-0.8509804 ]
  ...
  [-0.85882354]
  [-0.7647059 ]
  [-0.75686276]]

 [[-0.90588236]
  [-0.8745098 ]
  [-0.8980392 ]
  ...
  [-0.81960785]
  [-0.8039216 ]
  [-0.75686276]]

 [[-0.85882354]
  [-0.8901961 ]
  [-0.92156863]
  ...
  [-0.8666667 ]
  [-0.8980392 ]
  [-0.85882354]]

 ...

 [[-0.9529412 ]
  [-0.

In [ ]:
bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
cv2.imwrite('../images/out_original.png', bgr_image)

True